In [6]:
import pandas as pd

df = pd.read_csv("data3_usage_info(hours).csv", encoding='CP949')

r_df = pd.read_csv("3. 공공자전거 대여소 정보(22.06월 기준).csv", encoding='CP949')
r_df.drop([0,1,2,3], inplace=True)
r_df.rename(columns={"Unnamed: 4":"위도","Unnamed: 5":"경도"}, inplace=True)
r_df.rename(columns={"대여소\n번호":"번호", "보관소(대여소)명":"대여소 명","소재지(위치)":"소재지","Unnamed: 3":"주소", "설치\n시기":"설치시기"}, inplace=True)
r_df.isnull().sum()
r_df.loc[r_df['번호'].isna()]
r_df.drop([2657], inplace=True)
r_df['번호'] = r_df['번호'].astype(int)
r_df['번호'].dtype




,대여소\n번호,보관소(대여소)명,소재지(위치),Unnamed: 3,위도,경도,설치\n시기,설치형태,Unnamed: 8,운영\n방식,Unnamed: 10
4,102,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293,2022-05-03,NaN,15,QR,1.0
5,103,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353,2022-05-03,NaN,14,QR,1.0
6,104,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857,2022-05-12,NaN,13,QR,1.0
7,105,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254,2015-09-06,NaN,5,QR,1.0
8,106,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265,2022-05-12,NaN,12,QR,1.0


In [39]:
from geopy.geocoders import Nominatim
from tqdm import tqdm

r_df['동'] = None

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

for i, j in tqdm(r_df.iterrows()):
    g = geocoding_reverse((j['위도'], j['경도']))
  
    a_list = g[0].split(',')
    for c in range(1,len(a_list)):
        if a_list[c][-1] == '구' :
            if a_list[c-1][-1] == '동':
                dong = a_list[c-1].strip()
            else:
                dong ='없음'

    r_df['동'][i] = dong



0it [00:00, ?it/s]C:\Users\smdhk\AppData\Local\Temp\ipykernel_11580\848024442.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df['동'][i] = dong
2653it [25:14,  1.75it/s]


In [40]:
r_df.to_csv("r_df.csv", index = False, encoding='CP949')

In [42]:
# 동 주소 매핑 안 된 것 20개 -> 일일히 매핑해주

r_df[r_df['동']=='없음'].count()

번호             20
대여소 명          20
소재지            20
주소             20
위도             20
경도             20
설치시기           20
설치형태           10
Unnamed: 8     10
운영\n방식         20
Unnamed: 10    20
동              20
dtype: int64

In [51]:
k = r_df[r_df['동'] == '없음']
k

,번호,대여소 명,소재지,주소,위도,경도,설치시기,설치형태,Unnamed: 8,운영\n방식,Unnamed: 10,동
479,672,대광고등학교,동대문구,동대문구 신설동 397-2,37.57875061,127.024025,2020-03-27,NaN,17,QR,1.0,없음
623,866,브라운스톤 남산아파트,용산구,후암동 458,37.551548,126.9768601,2020-03-30,NaN,10,QR,1.0,없음
978,1352,e 편한세상 보문아파트 내,성북구,서울특별시 성북구 낙산길 255,37.58388138,127.0170975,2017-11-13,8,NaN,LCD,1.0,없음
981,1355,보문2교,성북구,서울특별시 성북구 보문로30길 29,37.58869934,127.0189133,2017-12-22,25,NaN,LCD,1.0,없음
987,1362,보문역6번출구 앞,성북구,보문로 12,37.58519745,127.0196686,2021-05-03,NaN,12,QR,1.0,없음
1385,2013,장승배기역 5번출구,동작구,서울특별시 동작구 상도로 지하 188,37.50585175,126.9393616,2017-05-12,30,NaN,LCD,1.0,없음
1390,2025,흑석역 1번출구,동작구,서울특별시 동작구 현충로 지하 90,37.50938034,126.9636154,2017-05-12,20,NaN,LCD,1.0,없음
1391,2026,흑석역 4번출구,동작구,서울특별시 동작구 현충로 지하 90,37.50858688,126.9634934,2017-07-05,10,NaN,LCD,1.0,없음
1397,2037,동작역 5번출구 동작주차공원,동작구,서울특별시 동작구 현충로 지하 220,37.50318909,126.97715,2017-05-12,20,NaN,LCD,1.0,없음
1398,2038,동작역 7번출구,동작구,서울특별시 동작구 현충로 지하 220,37.50318146,126.9760895,2017-05-12,10,NaN,LCD,1.0,없음


In [49]:
r_df['동'][479]

'없음'

In [52]:
r_df['동'][479] = '신설동'
r_df['동'][623] = '후암동'
r_df['동'][978] = '보문동'
r_df['동'][981] = '안암동'
r_df['동'][987] = '보문동'
r_df['동'][1385] = '상도동'
r_df['동'][1390] = '흑석동'
r_df['동'][1391] = '흑석동'
r_df['동'][1397] = '동작동'
r_df['동'][1398] = '동작동'
r_df['동'][1399] = '흑석동'
r_df['동'][1409] = '흑석동'
r_df['동'][1411] = '흑석동'
r_df['동'][1421] = '흑석동'
r_df['동'][1425] = '흑석동'
r_df['동'][1433] = '흑석동'
r_df['동'][2276] = '흑석동'
r_df['동'][2278] = '노량진동'
r_df['동'][2397] = '풍납동'
r_df['동'][2480] = '후암동'

C:\Users\smdhk\AppData\Local\Temp\ipykernel_11580\590452497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df['동'][479] = '신설동'
C:\Users\smdhk\AppData\Local\Temp\ipykernel_11580\590452497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df['동'][623] = '후암동'
C:\Users\smdhk\AppData\Local\Temp\ipykernel_11580\590452497.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df['동'][978] = '보문동'
C:\Users\smdhk\AppData\Local\Temp\

In [53]:
r_df[r_df['동']=='없음'].count()

번호             0
대여소 명          0
소재지            0
주소             0
위도             0
경도             0
설치시기           0
설치형태           0
Unnamed: 8     0
운영\n방식         0
Unnamed: 10    0
동              0
dtype: int64

In [54]:
r_df.to_csv("r_df2.csv", index = False, encoding='CP949')

In [55]:
df2 = pd.merge(df, r_df, left_on = '대여소번호', right_on='번호', how='left')

In [65]:
len(df) == len(df2)

True

In [63]:
r_df2 = r_df[['소재지','번호','위도','경도','동']]
r_df2.to_csv("동,소재,위도,경도.csv", index = False, encoding='CP949')